# Prediction of toxicity of small molecules

This notebook contains a end to end project for toxicity prediction of small molecules. Specific details about the code can be found in the present notebbok or in the helper scripts referenced in each section. 

For this project the dataset used was [MolToxPred](https://pubs.rsc.org/en/content/articlelanding/2024/ra/d3ra07322j). For further information check out the linked reference.

In [1]:
import pandas as pd
from collections import Counter

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/FabioHerrera97/Cheminformatics_ML_toxicity/refs/heads/main/data/smiles_10449_train_test.csv')
data.head()

,SMILES,Toxicity
0,Cn1cnc2c(F)c(Nc3ccc(Br)cc3Cl)c(C(=O)NOCCO)cc21,0
1,COC(=O)c1ccc2c(c1)NC(=O)/C2=C(\Nc1ccc(N(C)C(=O...,0
2,CN(C)C/C=C/C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2...,0
3,COC1CC2CCC(C)C(O)(O2)C(=O)C(=O)N2CCCCC2C(=O)OC...,0
4,CS(=O)(=O)O.Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc...,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10449 entries, 0 to 10448
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   SMILES    10449 non-null  object
 1   Toxicity  10449 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 163.4+ KB


In [4]:
print(data['Toxicity'].value_counts())

Toxicity
0    5833
1    4616
Name: count, dtype: int64


In [5]:
mol_counts = Counter(data['SMILES'])
duplicates = {element: count for element, count in mol_counts.items() if count > 1}
print(duplicates)

{}


This is a dataset containig 10449 compounds. There is a small imbalance as 5833 of the molecules are non-toxic (label 0), while the remaining 4616 are toxic (label 1)

## Standardization of the compounds

**NOTE: This section of the project is based on [DeepMol](https://deepmol.readthedocs.io/en/latest/) standardization tutorial and [MolPipeline](https://pubs.acs.org/doi/10.1021/acs.jcim.4c00863) example notebooks with some minor modifications.**

Standardization referes to transforming a set of chemical structures to a standardized format using a predifined set of rules. This allows to properly compare the chemical structures in the dataset to each other and handle steps like duplicated element deletion or ensure data consistency. 

There are 3 common standardization options: basic standardizer, complex standardizerand ChEMBL standardizer. Simple standardizer only perform sanititization, including steps like kekulize, check valencies, set aromaticity, conjugation and hybridization. Complex standardizers include customized procedures by performing additional steps like remove isotope information, neutralize charges, remove stereochemistry or remove smaller fragments. Finally, [ChEMBL](https://jcheminf.biomedcentral.com/articles/10.1186/s13321-020-00456-1) standardizer formats compounds according to defined rules and conventions.

In [6]:
from molpipeline import Pipeline
from molpipeline.any2mol import AutoToMol
from molpipeline.mol2mol import ElementFilter, MetalDisconnector, SaltRemover, StereoRemover, SolventRemover, TautomerCanonicalizer, Uncharger

In [7]:
#Create elements from smiles
mol_from_smiles = [('auto2mol', AutoToMol())]

#Set up the molecular standardization steps
standardization_steps = [
    ('metal_disconnector', MetalDisconnector()),
    ('salt_remover', SaltRemover()),
    ('element_filter', ElementFilter()),
    ('uncharge', Uncharger()),
    ('canonical_tautomer', TautomerCanonicalizer()),
    ('stereo_remover', StereoRemover())
]

pipeline_standardization = Pipeline(
    mol_from_smiles + standardization_steps
)

standardized_structures = pipeline_standardization.transform(data['SMILES'])

The above pipeline, in the standardization steps, include the following procedures:
   
1. `metal_disconnector`: remove metal atoms. Disconnecting metal ensures that the core organic molecules are properly analyzed as metals re not well-represented in cheminformatics.

2. `salt_remover`: Removes salt counterions (e.g., Na+, Cl-, K+) from the molecule. These ions are often added during synthesis or purification but are not part of the active molecule. These salts can artificially inflate the molecular weight or alter properties like solubility, leading to incorrect predictions and are irrelevant to the biological activity.

3. `element_filter`: In this step molecules containing elements other than the default elements (H, B, C, O, F, Si, P, S, Cl, Se, Br, I) are replaced with an `InvalidInstance` to instead of removing it and avoid inconsistencies with the initial input. This element filter is needed to avoid feature representation problems as unusual elements may not be properly encoded, they may also introduce noise or irrelevant patterns, and disturb the capacity of the model to capture meaningful biological information. **It is important to apply this filter only after disconnecting metals aor removing salts to avoid removing compounds of interest**

5. `uncharge`: Neutralizes charged molecules by adjusting protonation states (e.g., converting -COO⁻ to -COOH or -NH₃⁺ to -NH₂). Charged molecules can have different physicochemical properties (e.g., solubility, reactivity) compared to their neutral forms. Uncharging ensures consistency in molecular representation.

6. `canonical_tautomer`: Converts the molecule to a standardized tautomeric form. Tautomers are isomers that can interconvert by the movement of a proton and a double bond (e.g., keto-enol tautomerism). Tautomers can have different chemical properties, but they represent the same molecule in equilibrium. Without standardization, the same molecule could be represented in multiple ways, leading to inconsistencies in predictions, affecting the model reliability.

7. `stereo_remover`: Removes stereochemical information (e.g., chiral centers, double bond stereochemistry) from the molecule. Stereochemistry can significantly affect molecular properties (e.g., biological activity), but if only a part of the training data include stereochemistry, considering it in the input can lead to incorrect predictions.

Additional standardazation steps can be added. Chech [RDKit standardization module documentation](https://www.rdkit.org/docs/source/rdkit.Chem.MolStandardize.html) or [MolVS documentation](https://molvs.readthedocs.io/en/latest/guide/standardize.html). Also, you can check this talk from [Greg Landrum](https://www.youtube.com/watch?v=eWTApNX8dJQ)

**Note: every standardization step must be carefully selected to avoid overcleaning the data and affecting the predictive power of the model**

In [9]:
data ['Standardized_structure'] = standardized_structures
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10449 entries, 0 to 10448
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   SMILES                  10449 non-null  object
 1   Toxicity                10449 non-null  int64 
 2   Standardized_structure  10449 non-null  object
dtypes: int64(1), object(2)
memory usage: 245.0+ KB


In [10]:
structure_counts = Counter(data['Standardized_structure'])
duplicates_structure = {structure: count for structure, count in structure_counts.items() if count > 1}
print(duplicates_structure)

{InvalidInstance(ElementFilter, Molecule contains forbidden chemical element.): 837}


For the porpuse of this project, after completing the standardization these molecular structures (up to this point stored as mol objects) should be converted to a canonical smiles. This is necessary because after standadization some structures were manipulated and cleaned and therefore the smiles will changed. Also, this is required for the feature extraction step when using Large Language Models (see next section). This canonicalization also guarantees that the smiles are unique and avoids the presence of duplicated structures. However, every toolkit, and sometimes different versions of the same software, different algorithm. Therefore, **the uniqueness of the canonical smiles can only be ensured by using the same tool and version if this step is required in the future.**

In [11]:
import rdkit
from rdkit import Chem
import numpy as np

In [12]:
print("RDKit version:", rdkit.__version__)

RDKit version: 2024.09.5


In [13]:
def mol_to_smiles(mol):
    """
    Converts an RDKit molecule object to its canonical SMILES representation.

    Parameters:
    -----------
    mol : rdkit.Chem.rdchem.Mol
        An RDKit molecule object to be converted to a SMILES string.

    Returns:
    --------
    canonical_smiles : str or np.nan
        The canonical SMILES string corresponding to the input molecule.
        If the conversion fails (e.g., due to an invalid molecule), `np.nan` is returned.
    """
    try:
        canonical_smiles = Chem.MolToSmiles(mol)
    except Exception as e:
        # If an error occurs (e.g., invalid molecule), return Na
        canonical_smiles = np.nan

    return canonical_smiles

In [14]:
canonical_smiles = [mol_to_smiles(mol) for mol in data['Standardized_structure']]
data ['Canonical_smiles'] = canonical_smiles
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10449 entries, 0 to 10448
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   SMILES                  10449 non-null  object
 1   Toxicity                10449 non-null  int64 
 2   Standardized_structure  10449 non-null  object
 3   Canonical_smiles        9611 non-null   object
dtypes: int64(1), object(3)
memory usage: 326.7+ KB


Finally, after the canonicalization the invalid entries (837) are removed from the dataset keeping both the canonical smiles and the mol objects for practical porpuses in the next step.

In [15]:
cleaned_data = data.dropna().reset_index(drop=True)
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9611 entries, 0 to 9610
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   SMILES                  9611 non-null   object
 1   Toxicity                9611 non-null   int64 
 2   Standardized_structure  9611 non-null   object
 3   Canonical_smiles        9611 non-null   object
dtypes: int64(1), object(3)
memory usage: 300.5+ KB


In [16]:
cleaned_data.head()

,SMILES,Toxicity,Standardized_structure,Canonical_smiles
0,Cn1cnc2c(F)c(Nc3ccc(Br)cc3Cl)c(C(=O)NOCCO)cc21,0,<rdkit.Chem.rdchem.Mol object at 0x145f0ab21620>,Cn1cnc2c(F)c(Nc3ccc(Br)cc3Cl)c(C(=O)NOCCO)cc21
1,COC(=O)c1ccc2c(c1)NC(=O)/C2=C(\Nc1ccc(N(C)C(=O...,0,<rdkit.Chem.rdchem.Mol object at 0x145f0ab21bc0>,COC(=O)c1ccc2c(C(=Nc3ccc(N(C)C(=O)CN4CCN(C)CC4...
2,CN(C)C/C=C/C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2...,0,<rdkit.Chem.rdchem.Mol object at 0x145f0ab220c0>,CN(C)CC=CC(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc...
3,COC1CC2CCC(C)C(O)(O2)C(=O)C(=O)N2CCCCC2C(=O)OC...,0,<rdkit.Chem.rdchem.Mol object at 0x145f0ab22660>,COC1CC2CCC(C)C(O)(O2)C(=O)C(=O)N2CCCCC2C(=O)OC...
4,CS(=O)(=O)O.Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc...,0,<rdkit.Chem.rdchem.Mol object at 0x145f0ab21710>,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...


In [17]:
from ipywidgets import interact, fixed 
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG

mols = cleaned_data['Standardized_structure']

@interact(idx=range(0, len(mols)), ms=fixed(mols))
def show_mol(idx, ms):
    d2d = rdMolDraw2D.MolDraw2DSVG(450, 400)
    d2d.drawOptions().prepareMolsBeforeDrawing=False
    d2d.DrawMolecule(ms[idx])
    d2d.FinishDrawing()
    return SVG(d2d.GetDrawingText())

interactive(children=(Dropdown(description='idx', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1…

In [20]:
cleaned_data.to_csv('../data/cleaned_data.csv', index=False)

## Molecular respresentation

**NOTE: This section of the project is based on [DeepMol](https://deepmol.readthedocs.io/en/latest/) featurization tutorial, [MolPipeline](https://pubs.acs.org/doi/10.1021/acs.jcim.4c00863) example notebooks and [Fernandez-Torras et al.](https://www.sciencedirect.com/science/article/pii/S1367593121001204#fig1) with some modifications.**

![features_image.png](https://ars.els-cdn.com/content/image/1-s2.0-S1367593121001204-gr1_lrg.jpg)

Extracting features from molecules is a common task in machine learning and can be divided into classical methods and data driven methods. 

**Classical Methods**: There are 4 different types of features: 0D, 1D, 2D, 3D, or 4D.
 
- 0D features are descriptors that describe the individual parts of the molecule together as a whole, such as the number of atoms, bond counts or the molecular weight.
- 1D features are descriptors that describe substructures in the molecule (e.g. molecular fingerprints).
- 2D features are descriptors that describe the molecular topology based on the graph representation of the molecules, e.g. the number of rings or the number of rotatable bonds.
- 3D features are descriptors geometrical descriptors that describe the molecule as a 3D structure.
- 4D features are descriptors that describe the molecule as a 4D structure. A new dimension is added to characterize the interactions between the molecule and the active site of a receptor or the multiple conformational states of the molecule, e.g. the molecular dynamics of the molecule.
 
Source : [Molecular Descriptors for Structure–Activity Applications: A Hands-On Approach](https://link.springer.com/protocol/10.1007/978-1-4939-7899-1_1)
 
Calculating 3D features requires the generation of 3D conformers, which can be computationally expensive for large molecules. In addition, some features may not be available for certain molecules, e.g. 3D features cannot be calculated for molecules that do not have a 3D structure. Some tools like DeepMol provide methods for generating compound 3D structures.

**Data driven methods**: These are descriptors based on deep learning, pretrained models and LLMs strategies that encode molecules into abstract latent spaces, representing molecular similarities as simple distance measures between numerical vectors. Furthermore, molecular descriptors have expanded beyond chemistry, integrating relevant biological data from heterogeneous bioactivity assays. 

There is currently a huge amount of these methods comming out every day. This work of [The WhiteLab](https://pubs.rsc.org/en/content/articlelanding/2025/sc/d4sc03921a) presents a critical review of these tools depending on the case. In this tutorial we will use MolFormer, MolBERT, and ChemBERTa as they are the state of the art encoding methods for property prediction.

In [38]:
from molpipeline.mol2any import MolToMorganFP, MolToMACCSFP
from featurization import LLMsEncoding

In [49]:
%%time
pipeline_morgan = Pipeline(
    [
        ('morgan2_2048', MolToMorganFP(n_bits=1024, radius=2, return_as='dense'))
    ]
)

morgan_matrix = pipeline_morgan.transform(cleaned_data['Standardized_structure'])

CPU times: user 339 ms, sys: 4 ms, total: 343 ms
Wall time: 342 ms


In [51]:
morgan_df = pd.DataFrame(morgan_matrix)

In [40]:
%%time
pipeline_maccs= Pipeline(
    [
        ('maccs', MolToMACCSFP(return_as='dense'))
    ],
)

MACCS_matrix = pipeline_maccs.transform(cleaned_data['Standardized_structure'])
 

CPU times: user 6.63 s, sys: 3.91 ms, total: 6.63 s
Wall time: 6.63 s


In [52]:
MACCS_df = pd.DataFrame(MACCS_matrix)

In [41]:
%%time
chemberta_embeddings = LLMsEncoding.featurize(cleaned_data['Canonical_smiles'], 'DeepChem/ChemBERTa-77M-MTR', 600)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9611/9611 [00:22<00:00, 419.01it/s]

CPU times: user 4min 35s, sys: 258 ms, total: 4min 35s
Wall time: 24.1 s


In [53]:
chemberta_df = pd.DataFrame(chemberta_embeddings)

In [42]:
%%time
molformer_embeddings = LLMsEncoding.featurize(cleaned_data['Canonical_smiles'], 'ibm/MolFormer-XL-both-10pct', 2362)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.     | 0/9611 [00:00<?, ?it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9611/9611 [04:02<00:00, 39.66it/s]


CPU times: user 48min 25s, sys: 1.47 s, total: 48min 27s
Wall time: 4min 4s


In [54]:
molformer_df = pd.DataFrame(molformer_embeddings)

In [43]:
%%time
selformer_embeddings = LLMsEncoding.featurize(cleaned_data['Canonical_smiles'], 'HUBioDataLab/SELFormer', 800)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.     | 0/9611 [00:00<?, ?it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9611/9611 [03:22<00:00, 47.45it/s]

CPU times: user 40min 30s, sys: 940 ms, total: 40min 31s
Wall time: 3min 23s


In [55]:
selformer_df = pd.DataFrame(selformer_embeddings)